In [96]:
import numpy as np 
import pandas as pd 
import time 
import timeit 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.stats import linregress
from itertools import combinations, permutations

In [2]:
dataset = pd.read_csv("dati_MM1.csv") #tanto vale andare di ordine 2
display(dataset) #non contiene i trigger che ormai non ci servono più (abbiamo il time pedestal e tutto)

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER,EVENT,TIME_PEDESTAL,DRIFT_TIME,X,COLUMN
0,13.0,1.897417e+09,445.0,24.0,11145.000000,3.0,4.0,1.0,11350.0,205.000000,11.038462,4
1,14.0,1.897417e+09,450.0,9.0,11257.500000,3.0,2.0,1.0,11350.0,92.500000,4.980769,4
2,16.0,1.897417e+09,452.0,17.0,11314.166667,3.0,1.0,1.0,11350.0,35.833333,1.929487,4
3,15.0,1.897417e+09,458.0,9.0,11457.500000,3.0,3.0,1.0,11350.0,107.500000,5.788462,4
4,89.0,1.897417e+09,300.0,27.0,7522.500000,2.0,4.0,1.0,7825.0,302.500000,16.288462,7
...,...,...,...,...,...,...,...,...,...,...,...,...
114790,19.0,1.920940e+09,1449.0,2.0,36226.666667,3.0,3.0,1.0,36075.0,151.666667,8.166667,5
114791,55.0,1.920940e+09,325.0,4.0,8128.333333,3.0,3.0,1.0,8225.0,96.666667,5.205128,14
114792,56.0,1.920940e+09,327.0,22.0,8193.333333,3.0,1.0,1.0,8225.0,31.666667,1.705128,14
114793,54.0,1.920940e+09,331.0,20.0,8291.666667,3.0,2.0,1.0,8225.0,66.666667,3.589744,14


se ho X, COLUMN e LAYER posso costruire XR e XL come column +- x in un modo layer dependent; questo significa che tutte le altre colonne del dataframe non mi servono più e pertanto me ne sbarazzo (ma in modo "safe", cioè con una copia del dataset). Devo tenere orbit ed event perché mi aiutano ad identificare l'appartenenza di un dato hit ad un dato evento; infine aggiungo le colonne che riempirò a breve per indicare la posizione assoluta con ambiguità non risolta.

In [51]:
df = dataset.copy() 
df = df[["ORBIT_CNT","LAYER","EVENT","X","COLUMN"]]#tengo orbit, event, layer, column, x riassegnando a df se stesso a cui accedo mediante la lista delle colonne di interesse
df["XR"] = np.nan
df["XL"] = np.nan
display(df)
#df.info(memory_usage="deep") #4.4 mb

,ORBIT_CNT,LAYER,EVENT,X,COLUMN,XR,XL
0,1.897417e+09,4.0,1.0,11.038462,4,NaN,NaN
1,1.897417e+09,2.0,1.0,4.980769,4,NaN,NaN
2,1.897417e+09,1.0,1.0,1.929487,4,NaN,NaN
3,1.897417e+09,3.0,1.0,5.788462,4,NaN,NaN
4,1.897417e+09,4.0,1.0,16.288462,7,NaN,NaN
...,...,...,...,...,...,...,...
114790,1.920940e+09,3.0,1.0,8.166667,5,NaN,NaN
114791,1.920940e+09,3.0,1.0,5.205128,14,NaN,NaN
114792,1.920940e+09,1.0,1.0,1.705128,14,NaN,NaN
114793,1.920940e+09,2.0,1.0,3.589744,14,NaN,NaN


Adesso devo calcolare XR e XL, cioè le colonne contenenti le posizioni in coordinate assolute a partire da "in basso a sinistra" in mm (con ambiguità). Questo calcolo era già stato fatto (ma senza salvarne il risultato) dentro la funzione di disegno nel notebook precedente; mi basta allora riciclare parti di quel codice.

In [53]:
def compute_xr_xl(hit) :
    dx = 42 #lunghezza di un rettangolo in mm

    #anche stavolta le serie danno problemi, gli array di numpy no!
    if hit.LAYER.to_numpy(dtype=int) % 2 != 0 : #mi viene meglio fare prima il caso senza traslazione
            hit["XR"] = dx * (hit.COLUMN - 1/2) + hit.X #stessa formula di prima +- X
            hit["XL"] = dx * (hit.COLUMN - 1/2) - hit.X
    else :
        hit["XR"] = dx * hit.COLUMN + hit.X #qui devo sommare dx/2 per tenere conto della traslazione dei layer 2 e 4 (nella nostra convenzione), quindi il -1/2 di prima si semplifica 
        hit["XL"] = dx * hit.COLUMN - hit.X 
    return hit

df = df.groupby(["ORBIT_CNT","EVENT","LAYER"],group_keys=False).apply(compute_xr_xl) #fissare orbit ed event mi dà ~4 hits; siccome abbiamo eliminato tempo fa le hits con più celle attive sullo stesso layer se raggruppo anche per layer posso stare certo di trovare un'unica hit da utilizzare nella funzione di cui sopra
display(df)

,ORBIT_CNT,LAYER,EVENT,X,COLUMN,XR,XL
0,1.897417e+09,4.0,1.0,11.038462,4,179.038462,156.961538
1,1.897417e+09,2.0,1.0,4.980769,4,172.980769,163.019231
2,1.897417e+09,1.0,1.0,1.929487,4,148.929487,145.070513
3,1.897417e+09,3.0,1.0,5.788462,4,152.788462,141.211538
4,1.897417e+09,4.0,1.0,16.288462,7,310.288462,277.711538
...,...,...,...,...,...,...,...
114790,1.920940e+09,3.0,1.0,8.166667,5,197.166667,180.833333
114791,1.920940e+09,3.0,1.0,5.205128,14,572.205128,561.794872
114792,1.920940e+09,1.0,1.0,1.705128,14,568.705128,565.294872
114793,1.920940e+09,2.0,1.0,3.589744,14,591.589744,584.410256


Per potere effettuare un fit mi serve anche il valore di Y (che è lo stesso per XR e XL). Per calcolarlo basta usare la stessa formula usata nella funzione per graficare il detector nel time pedestal, che dipende solo dal valore del layer

In [57]:
dy = 13 #altezza singolo rettangolo in mm
df["Y"] = dy * (df.LAYER-1/2)
display(df)

,ORBIT_CNT,EVENT,XR,XL,LAYER,Y
0,1.897417e+09,1.0,179.038462,156.961538,4.0,45.5
1,1.897417e+09,1.0,172.980769,163.019231,2.0,19.5
2,1.897417e+09,1.0,148.929487,145.070513,1.0,6.5
3,1.897417e+09,1.0,152.788462,141.211538,3.0,32.5
4,1.897417e+09,1.0,310.288462,277.711538,4.0,45.5
...,...,...,...,...,...,...
114790,1.920940e+09,1.0,197.166667,180.833333,3.0,32.5
114791,1.920940e+09,1.0,572.205128,561.794872,3.0,32.5
114792,1.920940e+09,1.0,568.705128,565.294872,1.0,6.5
114793,1.920940e+09,1.0,591.589744,584.410256,2.0,19.5


A questo punto posso alleggerirmi ancora tenendo solo orbit ed event (per identificare l'evento) + XR, XL e Y per il calcolo delle rette.

In [58]:
df = df[["ORBIT_CNT","EVENT","XR","XL","Y"]]
display(df)

,ORBIT_CNT,EVENT,XR,XL,Y
0,1.897417e+09,1.0,179.038462,156.961538,45.5
1,1.897417e+09,1.0,172.980769,163.019231,19.5
2,1.897417e+09,1.0,148.929487,145.070513,6.5
3,1.897417e+09,1.0,152.788462,141.211538,32.5
4,1.897417e+09,1.0,310.288462,277.711538,45.5
...,...,...,...,...,...
114790,1.920940e+09,1.0,197.166667,180.833333,32.5
114791,1.920940e+09,1.0,572.205128,561.794872,32.5
114792,1.920940e+09,1.0,568.705128,565.294872,6.5
114793,1.920940e+09,1.0,591.589744,584.410256,19.5


Adesso arriva il bello!! 
Supponiamo di avere un evento dotato di 4 hits (una per layer) per un totale di 8 punti candidati a costruire la nostra retta. Per ciascun livello dobbiamo scegliere un punto; dal momento che abbiamo due punti da scegliere da ciascun layer e 4 layer in totale questo significa che abbiamo 2^4=16 possibili combinazioni di punti. Se l'evento "funziona" una di queste combinazioni produrrà una retta che attraverserà in pieno TUTTI i punti con cui è stata costruita anziché solo alcuni; poiché ci sono degli errori sperimentali sui drift_times e quindi sulle XR/XL non possiamo aspettarci che la retta "giusta" attraversi esattamente i punti "giusti", quindi dobbiamo piuttosto cercare la retta di best fit che minimizza la distanza fra i punti e la retta di best fit stessa.  
Volendo lavorare a forza bruta possiamo procedere così: 
- mediante meshgrid, transpose e reshape costruiamo in modo molto efficiente le 16 possibili combinazioni menzionate sopra. Come/perché questo funzioni è un po' difficile da spiegare; alla fine ci sono delle celle che ho usato per fare dei test che dovrebbero contenere un abbozzo di spiegazione, ma per il momento mi mantengo sul vago. L'alternativa è usare itertools.combinations, che pur essendo molto più intuitivo pare sia non altrettanto performante e mealleabile ai nostri scopi.
- per ciascuna di queste 16 combinazioni calcoliamo la retta di best fit mediante scipy.stats.linregress(); pare che questa sia la funzione che non ci richieda di costruire esplicitamente una matrice da invertire più veloce per effettuare la regressione lineare (stando a [questo articolo](https://www.freecodecamp.org/news/data-science-with-python-8-ways-to-do-linear-regression-and-measure-their-speed-b5577d75f8b/), perlomeno). Per misurare quella che si adatta meglio guardiamo al coefficiente di correlazione lineare r e scegliamo quella con r massimo; poiché non mi va di sprecare memoria per rette quasi tutte da buttare penso sia una buona idea tenere traccia della retta migliore e ogni volta confrontare con la successiva e scartare la retta sconfitta (si possono provare alternative, comunque. Magari salvandole e poi massimizzando r viene meglio, boh). Chiaramente oltre alla retta migliore salviamo quale combinazione di punti l'abbia prodotta!
- in linea di principio fatto questo lavoro dobbiamo controllare quale sia la massima distanza orizzontale di un qualunque punto selezionato dalla retta stessa. Sappiamo che nel caso di eventi buoni tale distanza è minore dell'incertezza sperimentale data da v_drift*25 ns; se questa condizione non si verifica vuol dire che la misura in questione non è compatibile con una traiettoria rettilinea che attraversi tutti i punti osservati (entro l'incertezza sperimentale) in nessun caso, e quindi l'evento va scartato.  
  
Ovviamente l'approccio di cui sopra è a forza bruta e probabilmente come prestazioni fa schifo, quindi mi limito a lavorare per il momento con un mini dataframe se non altro per arrivare a qualche risultato; oltre a scrivere le cose in modo più efficiente ha senso pensare ad un algoritmo che non abbia bisogno di costruire tutte e 16 le rette. Come detto la costruzione di tutte le coppie possibili mediante meshgrid è molto veloce, quindi è ragionevole aspettarsi che il grosso del tempo venga impiegato dai fit e dai confronti; per questo è desiderabile adottare una strategia più furba. Per ora vado a forza bruta incrociando le dita  

NOTA EXTRA: meshgrid mi è fallito per il momento, uso itertools.combinations

In [64]:
df_mini = df[df.ORBIT_CNT == df.ORBIT_CNT.unique()[0]]
display(df_mini)

,ORBIT_CNT,EVENT,XR,XL,Y
0,1.897417e+09,1.0,179.038462,156.961538,45.5
1,1.897417e+09,1.0,172.980769,163.019231,19.5
2,1.897417e+09,1.0,148.929487,145.070513,6.5
3,1.897417e+09,1.0,152.788462,141.211538,32.5


In [ ]:
def find_trajectory(event) :
    #v = np.array(np.meshgrid(event.XR.to_numpy(),event.XL.to_numpy())).T.reshape((-1,2)) #le
    # somo stanco

In [94]:
#print(list(combinations(df_mini[["XR","XL"]].values,4)))
#list1 = 

#all_combinations = [list(zip(each_permutation, list2)) for each_permutation in permutations(list1, len(list2))]

[(array([179.03846154, 156.96153846]), array([172.98076923, 163.01923077]), array([148.92948718, 145.07051282]), array([152.78846154, 141.21153846]))]


In [81]:
v = np.array(np.meshgrid(df_mini.XR.to_numpy(),df_mini.XL.to_numpy())).T#.reshape((-1,2))
print(v)

[[[179.03846154 156.96153846]
  [179.03846154 163.01923077]
  [179.03846154 145.07051282]
  [179.03846154 141.21153846]]

 [[172.98076923 156.96153846]
  [172.98076923 163.01923077]
  [172.98076923 145.07051282]
  [172.98076923 141.21153846]]

 [[148.92948718 156.96153846]
  [148.92948718 163.01923077]
  [148.92948718 145.07051282]
  [148.92948718 141.21153846]]

 [[152.78846154 156.96153846]
  [152.78846154 163.01923077]
  [152.78846154 145.07051282]
  [152.78846154 141.21153846]]]


In [86]:
print(df_mini[["XR","XL"]].values)

[[179.03846154 156.96153846]
 [172.98076923 163.01923077]
 [148.92948718 145.07051282]
 [152.78846154 141.21153846]]


In [89]:
v = (np.meshgrid(df_mini[["XR","XL"]].values))
print(v)

[array([179.03846154, 156.96153846, 172.98076923, 163.01923077,
       148.92948718, 145.07051282, 152.78846154, 141.21153846])]


In [11]:
 a = np.array([1,2,3])
 b = np.array([4,5,6])
 c = np.array(np.meshgrid(a,b))
 print(c)
 print("ok")
 print(c.T)
 print("okk")
 print(c.shape)
 print(c.T.shape)

[[[1 2 3]
  [1 2 3]
  [1 2 3]]

 [[4 4 4]
  [5 5 5]
  [6 6 6]]]
ok
[[[1 4]
  [1 5]
  [1 6]]

 [[2 4]
  [2 5]
  [2 6]]

 [[3 4]
  [3 5]
  [3 6]]]
okk
(2, 3, 3)
(3, 3, 2)


https://stackoverflow.com/questions/32034237/how-does-numpys-transpose-method-permute-the-axes-of-an-array 
 
La trasposizione senza tuple specificata in questo caso (3 assi) scambia il primo asse con l'ultimo (mantenendo fissato quello in mezzo). Inizialmente spostarsi lungo l'asse 0 significa saltare dal primo array 3x3 al secondo.  
Se mi trovo in x[0,0,:] e passo a x[1,0,:] (sto mantenendo fissato il secondo indice) significa che passo dalla prima riga della prima matrice ([1,2,3]) alla prima della seconda ([4,5,6]); siccome dopo la trasposizione questo deve essere equivalente a spostarsi lungo l'ultima dimensione questo vuol dire che nell'array finale lo stesso salto equivale a passare da x[:,0,0] a x[:,0,1] (coerentemente col fatto che evidentemente ho scambiato gli stess indici tenendo fissato il secondo) - cioè a saltare dalla prima colonna della prima matrice alla seconda colonna della stessa. Allora nell'array finale avrò come x.T[:,0,0] quello che prima era x[0,0,:], cioè [1,2,3]. Analogamente x.T[0,0,:] deve essere uguale a x[:,0,0], cioè [[1],[4]]. Continuando così penso si spieghi il risultato ottenuto sopra che evidentemente contiene tutte le coppie che cerco costruite in modo molto efficiente  
VEDI TABLET PER UN DISEGNO CHIARIFICATORE (e in generale boh, reinterpreta meglio)

In [18]:
#https://stackoverflow.com/questions/1208118/using-numpy-to-build-an-array-of-all-combinations-of-two-arrays pare funzionare anche con più di 2 array dentro meshgrid
a = np.array([-1,1])
b = 2*a 
c = 3*a 
d = 4*a

for i,j in zip([a,b,c,d],["a","b","c","d"]) :
    print("%s:"%j,i)
e = np.array(np.meshgrid(a,b,c,d)).T
print("\n")
print("e:",e) #ha effettivamente 2^4=16 array di 4 elementi (cioè 16 combinazioni)

a: [-1  1]
b: [-2  2]
c: [-3  3]
d: [-4  4]


e: [[[[[-1 -2 -3 -4]
    [-1  2 -3 -4]]

   [[ 1 -2 -3 -4]
    [ 1  2 -3 -4]]]


  [[[-1 -2  3 -4]
    [-1  2  3 -4]]

   [[ 1 -2  3 -4]
    [ 1  2  3 -4]]]]



 [[[[-1 -2 -3  4]
    [-1  2 -3  4]]

   [[ 1 -2 -3  4]
    [ 1  2 -3  4]]]


  [[[-1 -2  3  4]
    [-1  2  3  4]]

   [[ 1 -2  3  4]
    [ 1  2  3  4]]]]]
